In [1]:
import os

In [2]:
%pwd

'/Users/ntchindagiscard/Documents/end-end-mlflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/ntchindagiscard/Documents/end-end-mlflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unsizp_dir: Path


In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
            ) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unsizp_dir = config.unsizp_dir
        )

        return data_ingestion_config
        

In [8]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size


In [9]:
#Components
class Dataingestion:
    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config

    def download_file(self) -> None:

        if not os.exist(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            ) 
            logger.info(f"File download! With the following information : \n{headers} ")
        else:
            logger.info(f"File already exist of size {get_size(Path(self.config.local_data_file))}")
    
    def extrat_zip_file(self):
        unzip_path = self.config.unsizp_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [12]:
#data injestion pipeline
try:
    config = ConfigurationManager()
    data_injestion_config = config.get_data_ingestion_config()
    data_ingestion = Dataingestion(config = data_injestion_config)
    data_ingestion.download_file()
    data_ingestion.extrat_zip_file()
except Exception as e:
    raise e

[2021-01-01 05:13:17,673: INFO: common: Yaml file : config/config.yaml loaded successfully]
[2021-01-01 05:13:17,716: INFO: common: Yaml file : params.yaml loaded successfully]
[2021-01-01 05:13:17,737: INFO: common: Yaml file : schema.yaml loaded successfully]
[2021-01-01 05:13:17,746: INFO: common: Created directory at: artifacts]


TypeError: isinstance() arg 2 must be a type, a tuple of types, or a union